In [577]:
# First set of the cryptopals challenges:
# https://cryptopals.com/sets/1

In [3]:
# Convert hex to base 64
import base64

ts = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'


def hex_to_base64(s):
    '''
    convert hex to base64
    input: hex string
    output: base64 string
    '''
    try:
        decode = bytes.fromhex(s)
        b64 = base64.b64encode(decode)

        return b64.decode()
    except Exception as e:
        return 'not a valid strring', e

# check if true:
h = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'
hex_to_base64(ts) == h

True

In [77]:
# Explanation:
#https://www.codechef.com/PRJRF14/problems/XORSN

# XOR is Exlusive Or. 

ts1 = '1c0111001f010100061a024b53535009181c'
ts2 = '686974207468652062756c6c277320657965'

import codecs

def fixed_XOR(s1, s2):
    '''
    input: two equal length buffers
    output: XOR combination.
    '''
    # convert strings to bytes
    s1 = bytes.fromhex(s1)
    s2 = bytes.fromhex(s2)

    # xor both strings as hex numbers
    xord = [x^y for x,y in zip(s1, s2)]
    sent = bytes(xord)

    xord = codecs.encode(sent, 'hex')
    return xord.decode("utf-8")        


h = '746865206b696420646f6e277420706c6179'

# check if true: 
fixed_XOR(ts1, ts2) == h

True

In [9]:
# single byte XOR cipher
from collections import Counter
import numpy as np

# the hex encoded string:
ts1 = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

# http://en.algoritmy.net/article/40379/Letter-frequency-English
freqs = {
    'A' : 0.08167,
    'B' : 0.01492, 
    'C' : 0.02202, 
    'D' : 0.0453,
    'E' : 0.12702,
    'F' : 0.02228,  
    'G' : 0.02015,
    'H' : 0.06094,
    'I' : 0.06966,
    'J' : 0.0153,
    'K' : 0.01292,
    'L' : 0.04025,
    'M' : 0.02406,
    'N' : 0.06749,
    'O' : 0.07507,
    'P' : 0.01929,
    'Q' : 0.00095,
    'R' : 0.05987,
    'S' : 0.06327,
    'T' : 0.09356,
    'U' : 0.02758,
    'V' : 0.00978,
    'W' : 0.02560,
    'X' : 0.00150,
    'Y' : 0.01994,
    'Z' : 0.00077,
    ' ' : 0.13000,
}



# Evaluate each output and chose the one with the best score. 
# done using Bhattacharyya distance
# https://en.wikipedia.org/wiki/Bhattacharyya_distance

def bhatt(s):
    '''
    input: sentence
    output: Bhattacharyya distance of a sentence. Value is between 0 and 1, which represents the 'Englishness' of a sentence.  Where 0 is not English and 1 is English.
    '''
    score = sum([freqs.get(byte.upper(), 0) for byte in s])
    return score

# iterate over each of the 256 bytes in combination with the string, and see which byte performs best.
def single_XOR(s):
    '''
    input: hex encoded string
    output: key of single XOR
    '''
    if isinstance(s, str):
        s1 = bytes.fromhex(s)
    else: 
        s1 = s

    f_score = 0
    f_key = ''
    f_sent = ''

    for i in range(256):
        key = [i] * len(s)
        
        xord = [x ^ y for x , y in zip(s1, key)]

        sent = bytes(xord)
        sent = "".join(map(chr, sent))

        z_key = chr(key[0])
         
        score = bhatt(sent)
        
        if score > f_score:
            f_score = score
            f_sent = sent
            f_key = z_key
        
    return f_score, f_sent, f_key


single_XOR(ts1)

(2.1565, "Cooking MC's like a pound of bacon", 'X')

In [10]:
import pandas as pd

# One of the 60-character strings in this file[4.txt] has been encrypted by single-character XOR.
data = pd.read_fwf('4.txt', header=None)

data['score'] = data[0].apply(lambda x: single_XOR(x)[0])
data['sent'] = data[0].apply(lambda x: single_XOR(x)[1])


winner = data.iloc[data['score'].argmax()]
print(winner)

0        7b5a4215415d544115415d5015455447414c155c46155f...
score                                               2.0802
sent                       Now that the party is jumping\n
Name: 170, dtype: object


In [81]:
lines = ["Burning 'em, if you ain't quick and nimble\n", "I go crazy when I hear a cymbal"]
sent = str("".join(lines))


# Encrypt it, under the key "ICE", using repeating-key XOR.
key = "ICE"
# In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.
rep_key = key * len(sent)


def repeat_key_XOR(s, keystring):
    '''
    input: sentence as string and a keystring 
    output: encrypted sentence
    '''
    s = bytearray(s, 'utf-8')
    
    keystring = bytes(keystring, 'utf-8')
    sent = bytes([x^y for x,y in zip(s, keystring)])


    return sent.hex()


# It should come out to:
result = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"

# check if code works: 
repeat_key_XOR(sent, rep_key) == result

True

In [12]:
# imports 
import numpy as np 
import base64

# There's a file[6.txt] here. It's been base64'd after being encrypted with repeating-key XOR.
# Decrypt it. Here's how:

# 1. let KEYSIZE be the length of the key; try values from 2 to (say) 40


# 2. Write a function to compute the edit distance / Hamming distance between two strings. (hamming distance is the number of differing bits)
# The distance between
d_test1 = b'this is a test'
# and 
d_test2 = b'wokka wokka!!!'
# is 37. Make sure this is correct before you proceed.

def ham_dist(s1, s2):
    '''
    input: two strings
    output: hamming distance of two strings
    '''
    xord = "".join([bin(x^y) for x,y in zip(s1, s2)])
    xord = sum([int(x) for x in xord if x == '1'])

    return xord

# testcase
assert ham_dist(d_test1, d_test2) == 37


# For each KEYSIZE, take the *first* KEYSIZE worth of bytes, and the *second* KEYSIZE worth of bytes, and find the distance between them.
# Normalize this result by deviding by KEYSIZE
data = open('6.txt').read()
decode = base64.b64decode(data)

def key_estimate(data, keysize): 

    chunks = [data[i:i+keysize] for i in range(0, len(data), keysize)]
 
    even = chunks[1::2]
    odd = chunks[::2] 

    dist = sum([ham_dist(x,y) / keysize for x, y in zip(even, odd)]) / len(chunks)
    return dist, keysize


def key_size(keys_range):
    short = 5000
    k_size = 0
    for KEYSIZE in range(1, keys_range):
        if key_estimate(decode, KEYSIZE)[0] < short:
            short = key_estimate(decode, KEYSIZE)[0]
            k_size = KEYSIZE

    return k_size

# keysize is 29
key_length = key_size(41)


# Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
def chunky_text(ciphertext, chunksize):
    chunked = [ciphertext[i:i+chunksize] for i in range(0, len(ciphertext), chunksize) if i < len(ciphertext) - chunksize]
    return chunked

chunked = chunky_text(decode, (key_length))

# Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
def transpose_list(l):
    l = [list(x) for x in chunked]
    l = np.array(l).T.tolist()
    return l 

transposed = transpose_list(chunked)

# Solve each block as if it was single-character XOR. You already have code to do this.
# For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.
vignere_key = ''.join([single_XOR(x)[2] for x in transposed])
print('Key: ', vignere_key)


# decrypt the ciphertext
def vignere(ciphertext, key):
    key = bytearray.fromhex(key.encode('utf-8').hex())
    decrypt_bytes = [b ^ key[i % len(key)] for i, b in enumerate(ciphertext)]
    plaintext = ''.join([chr(b) for b in decrypt_bytes])
    return plaintext

print('decoded text: ', vignere(decode, vignere_key))

Key:  Terminator X: Bring the noise
decoded text:  I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you

In [18]:
# AES in ECB mode
# The Base64-encoded content in this file[7.txt]
data = open('7.txt').read()
decode = base64.b64decode(data) 

#has been encrypted via AES-128 in ECB mode under the key
key = 'YELLOW SUBMARINE'

# Decrypt it. You know the key, after all. 
# Easiest way: use OpenSSL::Cypher and give it AES-128-ECB as cipher.

# DO THIS WITH CODE!
# You could obviously decrypt this using the OpenSSL command-line tool, but we're having you get tECB working in code for a reason. 
#You'll need it _a lot_ later on, and not just for attacking ECB. 

# https://techtutorialsx.com/2018/04/09/python-pycrypto-using-aes-128-in-ecb-mode/
from Crypto.Cipher import AES

obj = AES.new(key, AES.MODE_ECB)
plaintext = obj.decrypt(decode).decode('utf-8')
print(plaintext)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


In [19]:
# Detect AES in ECB mode
# In this[8.txt] file are a bunch of hex-encoded ciphertexts.
# One of them has been encrypted with ECB.
# Detect it.
# Remember that the problem with ECB is that it is stateless and deterministic; the same 16 byte plaintext block will always produce the same 16 byte ciphertext.

data = open('8.txt').read()
decode = base64.b64decode(data)

